In [29]:
import numpy as np
initial_list = ['가', '까', '나', '다', '따', '라', '마', '바', '빠', '사', '싸', '아', '자', '짜', '차', '카', '타', '파', '하']
medial_list = ['아', '애', '야', '얘', '어', '에', '여', '예', '오', '와', '왜', '외', '요', '우', '워', '웨', '위', '유', '으', '의', '이']
final_list = ['으', '윽', '윾', '윿', '은', '읁', '읂', '읃', '을', '읅', '읆', '읇', '읈', '읉', '읊', '읋', '음', '읍', '읎', '읏', '읐', '응', '읒', '읓', '읔', '읕', '읖', '읗']


def _compose_korean_char(initial, medial, final, initial_p=None, medial_p=None, final_p=None):
    # try:
    initial_index = initial_list.index(initial) if (initial is not None) and (initial in initial_list) else None
    medial_index = medial_list.index(medial) if (medial is not None) and (medial in medial_list) else None
    final_index = final_list.index(final) if (final is not None) and (final in final_list) else None
    
    # 자소가 1개 이하인 경우 => 그나마 가장 괜찮을 걸로 대체
    if [initial_index, medial_index, final_index].count(None) >= 2:
        grapheme = [initial, medial, final]
        p = [initial_p, medial_p, final_p]
        idx = np.array(p).argmax()
        return [grapheme[idx], p[idx]]
    
    # 자소가 2개 이상인 경우 => 자소가 아닌 녀석만 기본 자소로 사용
    else:
        initial_index = initial_index if initial_index is not None else 11
        medial_index = medial_index if medial_index is not None else 0
        final_index = final_index if final_index is not None else 0
        
    char_code = 44032 + (initial_index * 21 + medial_index) * 28 + final_index
    return [chr(char_code), sum([initial_p, medial_p, final_p])/3]
    # except:


def compose_korean_char(text):
    composed = [_compose_korean_char(*c) for c in text]
    char = "".join([x[0] for x in composed])
    conf = [x[1] for x in composed]
    return [char, conf]

In [2]:
import os

os.chdir("/home/code/PaddleOCR")
os.getcwd()

'/home/code/PaddleOCR'

In [31]:
from ppocr.utils.korean_compose import decompose_korean_char
from rapidfuzz.distance import Levenshtein

def grapheme_edit_dis(x, y):
    _x, _y = x, y
    x = x.replace(" ", "")
    y = y.replace(" ", "")
    if len(x) == 0 or len(y) == 0:
        if len(x) == len(y):
            return 1
        else:
            return 0
        
    x = decompose_korean_char(x)
    y = decompose_korean_char(y)
    
    # x = "".join(["".join(v) if len(set(v)) > 1 else v[0] for v in x])
    # y = "".join(["".join(v) if len(set(v)) > 1 else v[0] for v in y])
    
    x = "".join(["".join(v) for v in x])
    y = "".join(["".join(v) for v in y])
    
    # print(_x, _y, 1 - Levenshtein.normalized_distance(x, y))
    return Levenshtein.normalized_distance(x, y)

In [40]:
def str_comp(s1, s2):
    if len(s1) != len(s2):
        return False
    return all([a == b for a, b in zip(s1, s2)])

In [71]:
from PIL import Image
from pathlib import Path
import json
character_path = "/home/works/rec_grapheme_full_horizontal_C+FML_v1.0/inference_result"
# grapheme_path = "/home/works/rec_grapheme_full_horizontal_G/inference_result"
test_label_path = "/home/labelsets/aihub_rec_full_horizontal_90:05:05/test_label.txt"
# eval_label_path = "/home/labelsets/aihub_rec_full_horizontal_8:1:1/eval_label.txt"


with open(character_path) as f:
    c_list = [json.loads(line.rstrip("\n").split("\t")[1].replace("\'", "\"")) for line in f.readlines()]
# with open(grapheme_path) as f:
#     g_list = [line.rstrip("\n").split("\t") for line in f.readlines()]
with open(test_label_path) as f:
    l_list = [line.rstrip("\n").split("\t") for line in f.readlines()]
# with open(eval_label_path) as f:
#     l_list += [line.rstrip("\n").split("\t") for line in f.readlines()]

n = 0
composed_long = 0
composed_short = 0
character_long = 0
character_short = 0
fl, fs, sl, ss, thl, ths = 0,0,0,0,0,0
both_long = 0
both_short = 0

short, long = 0, 0


composed_true = 0
character_true = 0
ensemble_true = 0
acc_composed_g_edit = 0
acc_character_g_edit = 0
acc_ensemble_g_edit = 0


acc_composed_c_edit = 0
acc_character_c_edit = 0
acc_ensemble_c_edit = 0
for i, (pred, (path, l)) in enumerate(list(zip(c_list, l_list))):
    # if l != "꽈배기":
    #     continue
    
    pred["character"][0] = pred["character"][0].replace(" ", "")
    pred["composed"][0] = pred["composed"][0].replace(" ", "")
    pred["first"][0] = pred["first"][0].replace(" ", "")
    pred["second"][0] = pred["second"][0].replace(" ", "")
    pred["third"][0] = pred["third"][0].replace(" ", "")
    l = l.replace(" ", "")
    f, fp = pred["first"]
    s, sp = pred["second"]
    th, thp = pred["third"]
    
    length = max([len(x) for x in [f, fp, s, sp, th, thp]])
    for x in [fp, sp, thp]:
        while len(x) < length:
            x.append(0)
    
    while len(f) < length: f += " "
    while len(s) < length: s += " "
    while len(th) < length: th += " "    
    
    composed = compose_korean_char(list(zip(f, s, th, fp, sp, thp)))
    character = pred["character"]
    
    # if len(composed[0]) < len(character[0]):
    #     ensemble = character[0]
    # elif len(composed[0]) > len(character[0]):
    #     ensemble = composed[0]
    # else:
    #     ensemble = "".join([c if gp < cp+0.2 else g for g, gp, c, cp in zip(*composed, *character)])
    
    ensemble = "".join([c if gp < cp+0.35 else g for g, gp, c, cp in zip(*composed, *character)])
    
    acc_composed_g_edit += grapheme_edit_dis(composed[0], l)
    acc_character_g_edit += grapheme_edit_dis(character[0], l)
    acc_ensemble_g_edit += grapheme_edit_dis(ensemble, l)
    
    
    acc_composed_c_edit += Levenshtein.normalized_distance(composed[0], l)
    acc_character_c_edit += Levenshtein.normalized_distance(character[0], l)
    acc_ensemble_c_edit += Levenshtein.normalized_distance(ensemble, l)
    

    
    if not str_comp(composed[0], l):
        composed_true +=1
    
    if not str_comp(character[0], l):
        character_true +=1 
        
    if not str_comp(ensemble, l):
        ensemble_true +=1 
        
        
    # if len(composed) > len(l):
    #     composed_long += 1
    # if len(character) < len(l):
    #     character_short += 1
    # if len(character) > len(l):
    #     character_long += 1
    # if len(character) > len(l) and len(composed) > len(l):
    #     both_long += 1
    # if len(character) < len(l) and len(composed) < len(l):
    #     both_short += 1
        
    # if len(pred["first"][0]) < len(l):
    #     fs += 1
    # if len(pred["first"][0]) > len(l):
    #     fl += 1
    # if len(pred["second"][0]) < len(l):
    #     ss += 1
    # if len(pred["second"][0]) > len(l):
    #     sl += 1
    # if len(pred["third"][0]) < len(l):
    #     ths += 1
    # if len(pred["third"][0]) > len(l):
    #     thl += 1
    
    # if max([len(pred["first"][0]), len(pred["second"][0]), len(pred["third"][0])]) < len(l):
    #     short += 1
    #     # print(pred["first"][0], pred["second"][0], pred["third"][0])
    # if max([len(pred["first"][0]), len(pred["second"][0]), len(pred["third"][0])]) > len(l):
    #     long += 1
    
    # if max([len(character), len(composed)]) < len(l):
    #     short += 1
    # if max([len(character), len(composed)]) > len(l):
    #     long += 1
    
    
    # if pred["composed"][0] != l and pred["character"][0] == l:
    # if str_comp(composed[0], l) and not str_comp(pred["character"][0], l):
        
    #     # print(composed[0], pred)
    #     # for g, c in zip(composed[0], pred["character"][0]):
    #     #     print(g, c)
    #     #     print(g == c)
    #     Image.open("/home/datasets/"+path).resize((100, 100)).show()
    #     print("character", character)
    #     print("composed", composed)
    #     print("ensemble", ensemble)
    #     print("first", f, fp)
    #     print("second", s, sp)
    #     print("third", th, thp)
    #     print(l)
    #     print()
    #     n += 1
        
        
        
    #     if n == 30:
    #         break
  
print(character_true, composed_true, ensemble_true)


# print(1 - acc_composed_g_edit/len(c_list))
# print(1 - acc_character_g_edit/len(c_list))
# print(1 - acc_ensemble_g_edit/len(c_list))


# print(1 - acc_composed_c_edit/len(c_list))
# print(1 - acc_character_c_edit/len(c_list))
# print(1 - acc_ensemble_c_edit/len(c_list))
# print(composed_short, composed_long)
# print(character_short, character_long)
# print(both_short, both_long)
# print(fs, fl, ss, sl, ths, thl)
# print(short, long)
# both = []
# only_g = []
# only_c = []
# neither = []

# for i, ((path, c, cp), (path, g, gp), (path, l)) in enumerate(list(zip(c_list, l_list))):
    
#     # c = c.replace(" ", "")
#     # g = g.replace(" ", "")
#     # l = l.replace(" ", "")

#     if (c==l) and (g==l):
#         both.append([path, l, c, cp, g, gp])
#     elif (c==l) and (g!=l):
#         only_c.append([path, l, c, cp, g, gp])
#     elif (c!=l) and (g==l):
#         only_g.append([path, l, c, cp, g, gp])
#     else:
#         neither.append([path, l, c, cp, g, gp])
# print(f"Both: {len(both)}")
# print(f"Only_G: {len(only_g)}")
# print(f"Only_C: {len(only_c)}")
# print(f"Neither: {len(neither)}")

1403 4668 1377


In [34]:
"실내크리닝" == "실내크리닝"

True

In [69]:
max([len(character)])
character

['나리', [0.7384371757507324, 0.9844052195549011]]

In [70]:
fc = only_g + neither
fg = only_c + neither
tc = both+only_c
tg = both+only_g
total = both+only_c+only_g+neither

In [73]:
import os
os.chdir("/home/code/PaddleOCR")
from ppocr.utils.korean_compose import decompose_korean_char 
from rapidfuzz.distance import Levenshtein

x = "안하세요"
y = "안A하세요"

def grapheme_edit_dis(x, y):
    
    _x, _y = x, y
    # x = x.replace(" ", "")
    # y = y.replace(" ", "")
    if len(x) == 0 or len(y) == 0:
        if len(x) == len(y):
            return 1
        else:
            return 0
        
    x = decompose_korean_char(x)
    y = decompose_korean_char(y)
    
    # x = "".join(["".join(v) if len(set(v)) > 1 else v[0] for v in x])
    # y = "".join(["".join(v) if len(set(v)) > 1 else v[0] for v in y])
    
    x = "".join(["".join(v) for v in x])
    y = "".join(["".join(v) for v in y])
    
    # print(_x, _y, 1 - Levenshtein.normalized_distance(x, y))
    return Levenshtein.normalized_distance(x, y)
        
def get_grapheme_pred_label(x):
    return [[g, l] for path, l, c, cp, g, gp in x]

def get_character_pred_label(x):
    return [[c, l] for path, l, c, cp, g, gp in x]

def print_grapheme_edit_dis(x, title="no_title", weighted=False):
    if weighted:
        total_length = sum([len(x) for x, l in x])
        dist = sum([(1-grapheme_edit_dis(x, l))*len(x) for x, l in x])/total_length
    else:
        total_length = len(x)
        dist = sum([1-grapheme_edit_dis(x, l) for x, l in x])/total_length
    print(f"G_NED({title}): {dist}")
    
    
def print_character_edit_dis(x, title="no_title", weighted = False):
    if weighted:
        total_length = sum([len(x) for x, l in x])
        dist = sum([(1-Levenshtein.normalized_distance(x, l))*len(x) for x, l in x])/total_length
    else:
        total_length = len(x)
        dist = sum([1-Levenshtein.normalized_distance(x, l) for x, l in x])/total_length
    print(f"C_NED({title}): {dist}")
    
def print_edit_dis(x, title, way, metric, weighted=False):
    filter = {"grapheme": get_grapheme_pred_label, "character":get_character_pred_label}[way]
    print_func = {"G_NED":print_grapheme_edit_dis, "C_NED":print_character_edit_dis}[metric]
    x = filter(x)
    print_func(x, title, weighted)

weighted = False
print_edit_dis(fc, "fc", "character", "C_NED", weighted=weighted)
print_edit_dis(fg, "fg", "grapheme", "C_NED", weighted=weighted)
print_edit_dis(fc, "fc", "character", "G_NED", weighted=weighted)
print_edit_dis(fg, "fg", "grapheme", "G_NED", weighted=weighted)


print()
print_edit_dis(only_g, "only_false_c", "character", "C_NED", weighted=weighted)
print_edit_dis(only_c, "only_false_g", "grapheme", "C_NED", weighted=weighted)
print_edit_dis(only_g, "only_false_c", "character", "G_NED", weighted=weighted)
print_edit_dis(only_c, "only_false_g", "grapheme", "G_NED", weighted=weighted)

C_NED(fc): 0.5861136257034557
C_NED(fg): 0.5527618471329632
G_NED(fc): 0.8011649113334545
G_NED(fg): 0.7412616908725692

C_NED(only_false_c): 0.6954248310169366
C_NED(only_false_g): 0.6126675494477795
G_NED(only_false_c): 0.8315530619477997
G_NED(only_false_g): 0.8219025012451135


In [74]:
weighted = True
print_edit_dis(fc, "fc", "character", "C_NED", weighted=weighted)
print_edit_dis(fg, "fg", "grapheme", "C_NED", weighted=weighted)
print_edit_dis(fc, "fc", "character", "G_NED", weighted=weighted)
print_edit_dis(fg, "fg", "grapheme", "G_NED", weighted=weighted)

print()
print_edit_dis(only_g, "only_false_c", "character", "C_NED", weighted=weighted)
print_edit_dis(only_c, "only_false_g", "grapheme", "C_NED", weighted=weighted)
print_edit_dis(only_g, "only_false_c", "character", "G_NED", weighted=weighted)
print_edit_dis(only_c, "only_false_g", "grapheme", "G_NED", weighted=weighted)

C_NED(fc): 0.7494845332263865
C_NED(fg): 0.6644557161062923
G_NED(fc): 0.8164793490285529
G_NED(fg): 0.788504044300266

C_NED(only_false_c): 0.7730620912872819
C_NED(only_false_g): 0.6734744198453874
G_NED(only_false_c): 0.8619085713551367
G_NED(only_false_g): 0.8410235735371968


In [53]:
fc

[['aihub_rec/863/862113.png',
  '법정스님',
  '법정스닝',
  '0.8810518980026245',
  '법정스님',
  '0.9917083978652954'],
 ['aihub_rec/709/708917.png',
  '이정오세무회계',
  '이정요세무회계',
  '0.9577153921127319',
  '이정오세무회계',
  '0.9843010107676188'],
 ['aihub_rec/207/206638.png',
  '아침햇살한의원',
  '아침핫살한의원',
  '0.8937467932701111',
  '아침햇살한의원',
  '0.9438696106274923'],
 ['aihub_rec/9/8233.png',
  '쉔픽스',
  '픽스',
  '0.9987812042236328',
  '쉔픽스',
  '0.9954103231430054'],
 ['aihub_rec/294/293066.png',
  '로또노래클럽',
  '로또노랜클럽',
  '0.9697343707084656',
  '로또노래클럽',
  '0.9939243594805399'],
 ['aihub_rec/778/777337.png',
  '대왕꽈배기',
  '대왕배기',
  '0.9988199472427368',
  '대왕꽈배기',
  '0.9971548318862915'],
 ['aihub_rec/265/264609.png',
  '간판',
  '안환',
  '0.4680023193359375',
  '간판',
  '0.8069177071253458'],
 ['aihub_rec/41/40709.png',
  '우리동네스파게티',
  '우리동네스파게디',
  '0.908715009689331',
  '우리동네스파게티',
  '0.9220299124717712'],
 ['aihub_rec/336/335291.png',
  '꽐라',
  '라',
  '0.9994766116142273',
  '꽐라',
  '0.9939570228258768'],
 ['ai

In [76]:
for path, l, c, cp, g, gp in fg[:10]:
    Image.open(Path("/home/datasets/")/path).resize((100, 100)).show()
    print(g, l, gp)



In [13]:
only_fg

[['한지', '한자'],
 ['돌이킬 ', '돌이킬 수'],
 ['셀로모텔', '첼로모텔'],
 ['상겹살데이', '삼겹살데이'],
 ['복방', '독방'],
 ['루시드아', '루시드'],
 ['필도', '팔도'],
 ['거강', '건강'],
 ['삼소나엄', '삼오나염'],
 ['듀통', '유통'],
 ['스칼레', '스칼렛'],
 ['베트남음식 전문정', '베트남음식 전문점'],
 ['의워', '의원'],
 ['잔치국쉬', '잔치국수'],
 ['포산심옹', '포산신용'],
 ['오륙도자복', '오륙도 족발'],
 ['여기는 카페', '여기는 카페다'],
 ['볼가', '본가'],
 ['방문 패', '방문판매'],
 ['아이사트로', '아이 스토리'],
 ['미 청', '미천왕'],
 ['플라돈', '플라톤'],
 ['셀프인테리어', '셀프 인테리어'],
 ['윈조할매', '원조할매'],
 ['표', '골프'],
 ['지오헤어빌', '지오헤어뷰'],
 ['오늘통디', '오늘통닭'],
 ['현래포', '현대포'],
 ['치괴', '치과'],
 ['나는주워 온', '나는 주워 온'],
 ['깨가부동산', '미가부동산'],
 ['암산장', '암산짱'],
 ['봑라', '봐라'],
 ['미용재료 봑허', '미용재료 백화점'],
 ['에수님', '예수님'],
 ['이북극', '북극'],
 ['홍메이드 가점식', '홈메이드 가정식'],
 ['울기 못해 웃꼬 간', '울지 못해 웃고 간'],
 ['보여준나', '보여준 날'],
 ['고운닝', '고운님'],
 ['차', '침'],
 ['상횐', '상회'],
 ['일끔등산', '일꿈동산'],
 ['형제에어건', '형제에어컨'],
 ['애플 내라', '애플노래방'],
 ['삼성 통사 훌', '삼성 특송 화물'],
 ['돕니다', '됩니다'],
 ['셀러', '셀럽'],
 ['배뢰쌈밥고기', '바로쌈밥고기'],
 ['이', '피'],
 ['내 결에 있어쥐서고마워', '내 곁에 있어줘서 고마워'],
 ['세방사상', '세방상

In [2]:
import os

os.chdir("/home/code/PaddleOCR")
os.getcwd()

from ppocr.utils.korean_compose import compose_korean_char, decompose_korean_char


f = "가a나"
s = "오야"
th = "은은"
fp = [1, 1, 1]
sp = [1, 1]
thp = [1, 1]

print(compose_korean_char(f, s, th, fp, sp, thp, first_main=False))
print(compose_korean_char(f, s, th, fp, sp, thp, first_main=True))



가a나
오야
은은
[1, 1]
[1, 1]
['곤얀', [1.0, 1.0]]
가a나
오 야
은 은
[1, 0, 1]
[1, 0, 1]
['곤a냔', [1.0, 1, 1.0]]
